<a href="https://colab.research.google.com/github/umarali92/umarali92/blob/main/Matching_With_Dropping_Condition.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import random

In [ ]:
data = pd.read_excel('TaskDataRefined.xlsx')
data['company_category_list'] = data['company_category_list'].str.split(",")
data = data[data['company_category_list'].notna()]
dict = {}
state_year = data.groupby(['company_state_code', 'founding_year']).count().reset_index()
data = data.sort_values(['max_activity_b'], ascending=False)
data['row_number'] = data.reset_index().index

In [ ]:
for index, row in state_year.iterrows():
    state = row["company_state_code"]
    year = row["founding_year"]
    if state not in dict:
        dict[state] = {}
    if year not in dict[state]:
        dict[state][year] = {}
    dict[state][year] = data[(data["company_state_code"] == state) & (data["founding_year"] == year)]

In [ ]:
visited_1 = set()
visited_2 = set()
identifier_1 = {}
identifier_2 = {}
count = 0

for index, row in data.iterrows():
    if row["max_activity_b"] == 1:
        state = row["company_state_code"]
        year = row["founding_year"]
        categories = row["company_category_list"]
        df = dict[state][year]
        matching_companies = df[df["company_domain"] != row["company_domain"]]
        matching_companies["number_matched"] = [len(set(categories).intersection(set(x))) for x in matching_companies['company_category_list']]
        group_1 = matching_companies[(matching_companies["number_matched"] > 0) & (~matching_companies['company_domain'].isin(visited_1))]["company_domain"].values.tolist()
        group_2 = matching_companies[(matching_companies["number_matched"] > 1) & (~matching_companies['company_domain'].isin(visited_2))]["company_domain"].values.tolist()
        if len(group_1) > 5:
          Group_1 = random.choices(group_1, k = 5)
        else:
          Group_1 = group_1[:5]
        visited_1.update(Group_1)
        if len(group_2) > 5:
          Group_2 = random.choices(group_2, k = 5)
        else:
          Group_2 = group_2[:5]
        visited_2.update(Group_2)
        data.loc[index,'group_1'] = ",".join(Group_1)
        data.loc[index,'group_2'] = ",".join(Group_2)
        identifier_1[row['company_domain']] = count
        identifier_2[row['company_domain']] = count
        for i in Group_1:
          identifier_1[i] = count
        for i in Group_2:
          identifier_2[i] = count
        count = count + 1

In [ ]:
for index, row in data.iterrows():
  data.loc[index, 'identifier_1'] = identifier_1[row['company_domain']] if row['company_domain'] in identifier_1 else None
  data.loc[index, 'identifier_2'] = identifier_2[row['company_domain']] if row['company_domain'] in identifier_2 else None

In [ ]:
df1 = data.drop(columns=['identifier_2', 'group_2'])
df2 = data.drop(columns=['identifier_1', 'group_1'])
df1.to_excel("Group1_With_Dropping_Condition.xlsx")
df2.to_excel("Group2_With_Dropping_Condition.xlsx")